In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset

c:\Users\salvi\anaconda3\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
data_path = "../data/devF.csv"
df = pd.read_csv(data_path)
df.head()

,Column1.id,Column1.url,Column1.category.category,Column1.category.title,Column1.text
0,0,https://onepiece.fandom.com/wiki/Ame_Ame_no_Mi,devil_fruits,Ame Ame no Mi,[ of Ame Ame no Mi]\n\n Ame Ame no Mi Statisti...
1,2,https://onepiece.fandom.com/wiki/Goro_Goro_no_Mi,devil_fruits,Goro Goro no Mi,[ of Goro Goro no Mi]\n\n Goro Goro no Mi Stat...
2,3,https://onepiece.fandom.com/wiki/Hie_Hie_no_Mi,devil_fruits,Hie Hie no Mi,[ of Hie Hie no Mi]\n\n Hie Hie no Mi Statisti...
3,4,https://onepiece.fandom.com/wiki/Magu_Magu_no_Mi,devil_fruits,Magu Magu no Mi,[ of Magu Magu no Mi]\n\n Magu Magu no Mi Stat...
4,1,https://onepiece.fandom.com/wiki/Gasu_Gasu_no_Mi,devil_fruits,Gasu Gasu no Mi,[ of Gasu Gasu no Mi]\n\n Gasu Gasu no Mi Stat...


In [3]:
len(df)

126

In [4]:
#df = df.replace(r'\n','', regex=True) 

In [5]:
df['Column1.text'][3]

"[ of Magu Magu no Mi]\n\n Magu Magu no Mi Statistics Japanese Name: マグマグの実 Official English Name: Mag-Mag Fruit (VIZ)Magma-Magma Fruit Meaning: Magma Usage Debut: Chapter 554; Episode 463 Type: Logia Current User: Sakazuki The Magu Magu no Mi is a Logia-type Devil Fruit that allows the user to create, control, and transform into magma at will, turning the user into a Magma Human (マグマ人間, Maguma Ningen?). It was eaten by Fleet Admiral Sakazuki, formerly known as Admiral Akainu. \nThe fruit's power was first shown during the Summit War of Marineford, but its name was not revealed until SBS Volume 62. "

In [6]:
import re

def extract_english_name(text):
    # Extract the English name using regex
    english_name = re.search(r'Official English Name:\s*([\w-]+)', text)
    english_name = english_name.group(1) if english_name else None
    
    # Remove text between square brackets and the English name section
    text = text.replace('\n', ' ').strip()
    cleaned_text = re.sub(r'\[.*?\]', '', text)
    cleaned_text = re.sub(r'Official English Name:\s*[\w-]+', '', cleaned_text)
    
    return cleaned_text.strip(), english_name

In [7]:
df['description'], df['english_name'] = zip(*df['Column1.text'].apply(extract_english_name))


In [8]:
df.drop(['Column1.id','Column1.url','Column1.category.category','Column1.text'],inplace=True,axis=1)

In [9]:
df['description'][0]

'Ame Ame no Mi Statistics Japanese Name: アメアメの実  Ame Fruit Meaning: Candy Usage Debut: Movie 4 Type: Logia Current User: Gasparde  The Ame Ame no Mi is a movie-only Logia Devil Fruit that allows the user to create, control, and transform into candy syrup at will. It was eaten by the main antagonist of the fourth movie, Gasparde.'

In [10]:
from bs4 import BeautifulSoup
class Cleaner():
    def __init__(self):
        pass 
    
    def put_line_breaks(self, text):
        return text.replace("<\p>", "<\p>\n")
    
    def remove_html_tags(self, text):
        clean_text = BeautifulSoup(text, "lxml").text
        return clean_text

    def clean(self, text):
        text = self.put_line_breaks(text)
        text = self.remove_html_tags(text)
        text = text.strip()
        return text

In [11]:
df.head()

,Column1.category.title,description,english_name
0,Ame Ame no Mi,Ame Ame no Mi Statistics Japanese Name: アメアメの実...,Ame
1,Goro Goro no Mi,Goro Goro no Mi Statistics Japanese Name: ゴロゴロ...,Rumble-Rumble
2,Hie Hie no Mi,Hie Hie no Mi Statistics Japanese Name: ヒエヒエの実...,Ice-Ice
3,Magu Magu no Mi,Magu Magu no Mi Statistics Japanese Name: マグマグ...,Mag-Mag
4,Gasu Gasu no Mi,Gasu Gasu no Mi Statistics Japanese Name: ガスガス...,Gas-Gas


In [12]:
df.rename(columns={'Column1.category.title': 'japanese_name'},inplace=True)

In [13]:
le = preprocessing.LabelEncoder()
le.fit(df['english_name'].tolist())

LabelEncoder()

In [15]:
label_dict = {index:label_name for index, label_name in enumerate(le.__dict__['classes_'].tolist())}


In [16]:
df['label'] = le.transform(df['english_name'].tolist())

In [17]:
df.head()

,japanese_name,description,english_name,label
0,Ame Ame no Mi,Ame Ame no Mi Statistics Japanese Name: アメアメの実...,Ame,0
1,Goro Goro no Mi,Goro Goro no Mi Statistics Japanese Name: ゴロゴロ...,Rumble-Rumble,78
2,Hie Hie no Mi,Hie Hie no Mi Statistics Japanese Name: ヒエヒエの実...,Ice-Ice,46
3,Magu Magu no Mi,Magu Magu no Mi Statistics Japanese Name: マグマグ...,Mag-Mag,53
4,Gasu Gasu no Mi,Gasu Gasu no Mi Statistics Japanese Name: ガスガス...,Gas-Gas,32


In [19]:
test_size = 0.2
df_train, df_test = train_test_split(df, 
                                     test_size=test_size, 
                                    )

In [20]:
model_name = "distilbert/distilbert-base-uncased"

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\salvi\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\salvi\anaconda3\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\salvi\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [22]:
def preprocess_function(tokenizer,examples):
    return tokenizer(examples['text_cleaned'],truncation=True)

In [ ]:
# Conver Pandas to a hugging face dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# tokenize the dataset
tokenized_train = train_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)
tokenized_test = test_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)